In [ ]:
# Import libraries
import ipynb
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as Data
import time
import os
import random
import matplotlib.pyplot as plt
import pdb

In [ ]:
# Import library functions from other .ipynb files 
from ipynb.fs.full.Data_MetarPPG import SlideWindowDataLoader, testing
from ipynb.fs.full.MainModel_MetarPPG import meta_rPPG
from ipynb.fs.full.Settings_MetarPPG import TrainOptions

In [ ]:
# Get the training options
opt = trainOptions().get_options()
num_iter = opt.batch_size

# Call and setup the model
model = meta_rPPG(opt, isTrain=True, continue_train=opt.continue_train)
model.setup(opt)

# Define the train and test sets
trainset= SlideWindowDataLoader(opt, isTrain=True)
testset= SlideWindowDataLoader(opt, isTrain=False)

index = opt.per_iter_task
trainset_size = trainset.num_tasks * (trainset.task_len[0] - opt.win_size)
task_len = trainset.task_len[0] - index*opt.win_size

iters = 0
print("Data Size: %d ||||| Batch Size: %d ||||| initial lr: %f" % (trainset_size, opt.batch_size, opt.lr))

tasklist = random.sample(range(5), opt.batch_size)
model.dataset = trainset
data = trainset[tasklist, 0]

# Set the input
model.set_input(data)
# Update the prototypical distance
model.update_prototype()
# Set the error metrics
mae = [10, 10]
rmse = [10, 10]
merate = [10, 10]
saving = 1

# Iterate through each task for each epoch
for epoch in range(opt.epach_count, opt.train_epoch+1):
    start_time = time.time()
    epoch_iter = 0
    i = 0
    
    for idx in range(0, task_len, 1):
        tasklist = random.sample(range(5), opt.batch_size)
        model.rppg.feed_hc([model.h, model.c])
        # Check progress
        model.progress = epoch + float(idx/task_len)
        
        for i in range(index):
            data = trainset[task_list, idx + i*opt.win_size]
            #iter_start_time = time.time()
            #total_iters += opt.win_size
            model.set_input(data)
            if i == 0:
                # Update the theta variable during the adaptation phase
                model.new_theta_update(epoch) 
            else:
                # Update the psi and phi variable during the learning phase
                model.new_psi_phi_update(epoch)
                
        # Get the test loss
        loss, testloss = testing(opt, model, testset, idx, epoch)
        
        epoch_iter += 1
        data = trainset[task_list, np.random.randint(task_len)]
        
        # Set the input
        model.set_input(data)
        # Update the prototypical distance
        model.update_prototype()
        
    # Save the system network and epoch
    # good clause in case of any accidents
    model.save_networks('latest')
    model.save_networks(epoch)
    
    # update the learning rate
    lr = model.update_learning_rate(epoch)
    print('Epoch %d/%d ||||| Time: %d sec ||||| Lr: %.7f ||||| Loss: %.3f/%.3f' % (epoch, opt.train_epoch, time.time() - epoch_start_time, new_lr, loss, test_loss))